In [2]:
import pandas as pd
from tensorflow import keras
import numpy as np
from small_functions import print_2d_data

In [3]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码

In [4]:
orig_data.iloc[0:4,]

,school,school_setting,school_type,classroom,teaching_method,n_student,student_id,gender,lunch,pretest,posttest
0,ANKYI,Urban,Non-public,6OL,Standard,20.0,2FHT3,Female,Does not qualify,62.0,72.0
1,ANKYI,Urban,Non-public,6OL,Standard,20.0,3JIVH,Female,Does not qualify,66.0,79.0
2,ANKYI,Urban,Non-public,6OL,Standard,20.0,3XOWE,Male,Does not qualify,64.0,76.0
3,ANKYI,Urban,Non-public,6OL,Standard,20.0,556O0,Female,Does not qualify,61.0,77.0


In [5]:
X.iloc[0:4,]

,n_student,pretest,school_setting_Rural,school_setting_Suburban,school_setting_Urban,school_type_Non-public,school_type_Public,teaching_method_Experimental,teaching_method_Standard,gender_Female,gender_Male,lunch_Does not qualify,lunch_Qualifies for reduced/free lunch
0,20.0,62.0,0,0,1,1,0,0,1,1,0,1,0
1,20.0,66.0,0,0,1,1,0,0,1,1,0,1,0
2,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
3,20.0,61.0,0,0,1,1,0,0,1,1,0,1,0


一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam

In [21]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率） 所以可以一开始把学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("偏差b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 645us/step - loss: 1061.1740
Epoch 2/100
54/54 [==============================] - 0s 529us/step - loss: 91.8851
Epoch 3/100
54/54 [==============================] - 0s 516us/step - loss: 58.2297
Epoch 4/100
54/54 [==============================] - 0s 516us/step - loss: 51.6138
Epoch 5/100
54/54 [==============================] - 0s 511us/step - loss: 45.0516
Epoch 6/100
54/54 [==============================] - 0s 512us/step - loss: 39.1323
Epoch 7/100
54/54 [==============================] - 0s 562us/step - loss: 33.8805
Epoch 8/100
54/54 [==============================] - 0s 587us/step - loss: 29.3810
Epoch 9/100
54/54 [==============================] - 0s 534us/step - loss: 25.7522
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 22.7832 - val_loss: 25.7436
Epoch 11/100
54/54 [==============================] - 0s 515us/step - loss: 20.4351
Epoch 12/100
54/54 [==============================] - 0s 547us/st

换个优化器 随机梯度下降

In [22]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(lr=0.01, clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("偏差b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 538us/step - loss: 4984.3325
Epoch 2/100
54/54 [==============================] - 0s 474us/step - loss: 2288.1069
Epoch 3/100
54/54 [==============================] - 0s 469us/step - loss: 667.4547
Epoch 4/100
54/54 [==============================] - 0s 538us/step - loss: 113.8209
Epoch 5/100
54/54 [==============================] - 0s 468us/step - loss: 61.7433
Epoch 6/100
54/54 [==============================] - 0s 546us/step - loss: 44.2219
Epoch 7/100
54/54 [==============================] - 0s 457us/step - loss: 31.1775
Epoch 8/100
54/54 [==============================] - 0s 476us/step - loss: 22.5707
Epoch 9/100
54/54 [==============================] - 0s 490us/step - loss: 17.8581
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 15.3927 - val_loss: 14.0964
Epoch 11/100
54/54 [==============================] - 0s 484us/step - loss: 14.3597
Epoch 12/100
54/54 [==============================] - 0s 461u